# Chain-of-Thought (CoT) Loan Application – Tools Needed Demo

In this demo, rather than completing a final "Approved" or "Denied" decision, our agent **only** determines which tools would be necessary to handle the loan application query.

Key changes from the original:
1. **LLM Adapter**: Instead of returning "Final Answer: Approved" or "Denied," it now **decides on the tools** to use.
2. **CoT Agent**: We still perform domain checks, but the final answer is **which tools** the agent believes it needs. We do **not** actually call them.

Let's walk through the code below.

## Section 1: Loan Criteria
We keep the same **policy** and **basic_loan_eligibility_check** as before, so we can see if the user meets certain criteria. This is just domain logic.

In [ ]:
# Loan criteria
LOAN_POLICY = {
    "MIN_CREDIT_SCORE": 650,       # Minimum acceptable credit score
    "MIN_ANNUAL_INCOME": 30000,   # Minimum annual income in dollars
    "MAX_DEBT_TO_INCOME": 0.4     # 40% maximum debt-to-income ratio
}

def basic_loan_eligibility_check(credit_score, annual_income, monthly_debt, monthly_income):
    """
    Returns a dict with high-level checks:
    {
      'credit_score_check': bool,
      'income_check': bool,
      'dti_check': bool
    }
    """
    if monthly_income > 0:
        dti = monthly_debt / monthly_income
    else:
        dti = 1.0  # Force fail if monthly_income is zero or negative

    results = {
        "credit_score_check": credit_score >= LOAN_POLICY["MIN_CREDIT_SCORE"],
        "income_check": annual_income >= LOAN_POLICY["MIN_ANNUAL_INCOME"],
        "dti_check": dti <= LOAN_POLICY["MAX_DEBT_TO_INCOME"]
    }
    return results

## Section 2: LLM Adapter
### Key Change
Here we modify the `call_llm` function. Instead of returning a final "Approved" or "Denied" decision, it returns:
1. A **Chain of Thought** describing the reasoning.
2. A **list of tools** the agent would want to invoke to handle the query.

For demonstration, we hardcode the logic:
- If all checks are true, maybe it **only** needs a "DocumentGenerator" tool to finalize paperwork.
- If the credit score is borderline, it might say we need a "CreditBureauAPI".
- If the income is too low, we might add a "CoSignerLookup" tool, etc.

In real usage, you’d parse the domain data dynamically and decide which tools to include.

In [ ]:
def call_llm(prompt, temperature=0.2):
    """
    Simulated LLM call. Instead of returning Approved/Denied,
    it returns which tools are needed based on the chain-of-thought.
    """
    # We'll simply check if certain lines are in the prompt (like 'credit_score_check: False')
    # and use that to decide which tools might be needed.

    if "Please reason step-by-step" in prompt:
        chain_of_thought = (
            "Chain of Thought:\n"
            "1. The agent looks at credit score, income, and debt-to-income ratio.\n"
            "2. If any fail, it might need more tools to verify or gather extra info.\n"
            "3. Otherwise, it might just finalize the normal procedure.\n"
"
        )

        # We'll do some naive string checks to see if any policy checks are false
        tools_needed = []

        if "credit_score_check: False" in prompt:
            tools_needed.append("CreditBureauAPI")
        if "income_check: False" in prompt:
            tools_needed.append("CoSignerLookup")
        if "dti_check: False" in prompt:
            tools_needed.append("DebtCounselingTool")

        # If no checks failed, maybe just need a standard doc generator
        if not tools_needed:
            tools_needed.append("DocumentGenerator")

        # Format final answer
        final_answer = "The tools to invoke are: " + ", ".join(tools_needed)

        response = chain_of_thought + "\n" + final_answer
        return response
    else:
        return "No chain of thought generated. (No tool decision)"

## Section 3: Chain-of-Thought Agent
We still use the **domain checks** from `basic_loan_eligibility_check`, but now the LLM is only deciding **which tools** we might want to use—**not** actually calling them.

The final dictionary returned includes:
- `chain_of_thought`: The step-by-step reasoning text from the LLM.
- `tools`: Which tools the LLM says are needed.


In [ ]:
class LoanCoTAgent:
    def __init__(self):
        self.conversation_log = []  # optional logging

    def process_loan_application(self, credit_score, annual_income, monthly_debt, monthly_income):
        """
        1. Run basic checks (domain logic).
        2. Prompt the LLM for chain-of-thought about which tools are needed.
        3. Parse and return the chain-of-thought + tools.
        """
        # Step 1: Basic checks
        eligibility_checks = basic_loan_eligibility_check(
            credit_score=credit_score,
            annual_income=annual_income,
            monthly_debt=monthly_debt,
            monthly_income=monthly_income
        )

        # Build a context string to help the LLM see which checks passed/failed
        context_info = (
            f"Applicant's credit score: {credit_score}\n"
            f"Applicant's annual income: {annual_income}\n"
            f"Applicant's monthly debt: {monthly_debt}\n"
            f"Applicant's monthly income: {monthly_income}\n"
            f"Checks:\n"
            f" - credit_score_check: {eligibility_checks['credit_score_check']}\n"
            f" - income_check: {eligibility_checks['income_check']}\n"
            f" - dti_check: {eligibility_checks['dti_check']}\n"
        )

        # Step 2: Prompt the LLM
        prompt = (
            "You are a loan approval AI. You have the following data:\n"
            f"{context_info}\n"
            "Please reason step-by-step about which tools are needed to process this loan fully. "
            "If any checks fail, we might need additional verification or partner tools.\n"
            "Your response should list a chain of thought and end with: \n"
            "'The tools to invoke are: [list of tools]'\n"
            "(No actual invocation needed.)\n"
        )

        llm_response = call_llm(prompt)
        self.conversation_log.append(llm_response)

        # Step 3: Extract which tools from LLM response
        # We'll look for a line that starts with 'The tools to invoke are:'
        chain_of_thought = llm_response
        tools_str = "None"

        if "The tools to invoke are:" in llm_response:
            tools_str = llm_response.split("The tools to invoke are:")[-1].strip()

        return {
            "chain_of_thought": chain_of_thought,
            "tools": tools_str
        }

## Section 4: Main Orchestration
We now **simulate** a loan application with certain data. When we run the notebook, the final output will show:
- The **chain-of-thought** from the LLM.
- Which **tools** it decided to invoke.

No actual tool calls will be made.

In [ ]:
# Example usage
if __name__ == "__main__":
    # Sample applicant data
    applicant_credit_score = 680
    applicant_annual_income = 28000.0  # intentionally below min
    applicant_monthly_debt = 800.0
    applicant_monthly_income = 3000.0

    # Initialize our Chain-of-Thought agent
    agent = LoanCoTAgent()

    # Process the loan application
    result = agent.process_loan_application(
        credit_score=applicant_credit_score,
        annual_income=applicant_annual_income,
        monthly_debt=applicant_monthly_debt,
        monthly_income=applicant_monthly_income
    )

    # Print both the chain of thought and the final 'tools'
    print("Chain of Thought + Tools:\n")
    print(result["chain_of_thought"], "\n")
    print(f"Tools: {result['tools']}")

## How to Run
1. **Run all cells** in the notebook.
2. Observe the **chain-of-thought** and the **list of tools** that the LLM decides upon.
3. Modify the applicant data to see how the tools might change (e.g., if the credit score is below 650, the LLM might add a “CreditBureauAPI”).

## Key Takeaways
1. **No Actual Tool Usage**: We only decide **which** tools might be needed—**no** real calls happen.
2. **Chain-of-Thought**: The LLM reasoning is still shown, but the final outcome is **tools** instead of "Approved" or "Denied."
3. **Easily Adaptable**: You can swap in different conditions or logic to show how the LLM might choose among a variety of possible tools.